In [21]:
import pandas as pd
import numpy as np
import random

# file_names = ['jpm', 'jp morgan', 'jpmorgan']
file_names = ['jpm']

data = pd.DataFrame()
for fName in file_names:
    data = data.append(pd.read_csv("reddit-data/" + fName + '.csv', sep=",", encoding="utf-8", error_bad_lines = False, header = 0))

In [22]:
data.shape

(30550, 6)

In [28]:
import html

def cleanRedditBody(df):
    data['body'] = data['body'].apply(lambda body: clean_tweets(html.unescape(body)))
    data['body'] = data[~data['body'].str.lower().str.contains('')]


# filterSubReddits(temp)
data = cleanRedditBody(data)
data.shape

SyntaxError: invalid syntax (<ipython-input-28-9d7c667044c1>, line 6)

In [29]:
data.to_csv("output/reddit-cleaning.csv",encoding='utf-8-sig',index=False)

In [19]:
def filterSubReddits(temp):
    subreddit_omit = ['formula', 'F1', "nascar", "AmericanHorrorStory"]

    for sub in subreddit_omit:
        temp = temp[~temp['subreddit'].str.lower().str.contains(sub, na = False)]
        
    return temp

In [24]:
import nltk
from nltk.corpus import stopwords
import re #regular expression
import string

#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

emoticons = emoticons_happy.union(emoticons_sad)

# Clean tweets from tweepy
def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    word_tokens = nltk.word_tokenize(tweet)
    #after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
    #remove urls
    tweet = re.sub(r'^https?:\/\/.*[\r\n]*', '', tweet, flags=re.MULTILINE)
    #filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
    #looping through conditions
    for w in word_tokens:
    #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet

In [6]:
#placeholder method => using Random number
def getSentiment(body):
#     body = clean_tweets(body)
    sent_list = [0,2,4]
    return (random).choice(sent_list)

In [7]:
def getRedditData(min_date, rolling_window):
    
    #   convert date str
    #     target_date = "2015-01-01"
    data['created'] = pd.to_datetime(data['created'], format='%Y/%m/%d',errors='coerce') 
    temp = data[data['created'] > min_date]
    
    temp = filterSubReddits(temp)
    
    for row in temp.itertuples():
        temp.loc[row.Index, 'created'] = row.created.date()
        temp.loc[row.Index, 'sentiment'] = getSentiment(row.body)

    #group by date and sentiment and unstack
    df = temp.groupby(['created','sentiment']).size().unstack(level=-1).fillna(0)
    #fill in missing time series with 0 values
    df = df.resample('D').mean().fillna(0)
    #generate rolling window, sort reverse
    df = df.rolling(rolling_window).sum()[::-1]
    
    return df

In [9]:
getRedditData('2016-01-01', 2)

sentiment,0.0,2.0,4.0
created,,,
2019-06-26,26.0,31.0,36.0
2019-06-25,26.0,24.0,35.0
2019-06-24,8.0,7.0,5.0
2019-06-23,8.0,10.0,8.0
2019-06-22,8.0,11.0,12.0
2019-06-21,8.0,10.0,13.0
2019-06-20,10.0,17.0,15.0
2019-06-19,14.0,22.0,13.0
2019-06-18,14.0,14.0,11.0


In [14]:
string = "&gt;"

import html
print(html.unescape(string))

>
